In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as snsfrom 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

In [81]:
austria_df = pd.read_csv(f'../data/Processed/time_series_processed_2.csv')
austria_df.set_index('utc_timestamp')

,Unnamed: 0,load_actual,load_forecast,price,solar_generation,Wind_generation,Weekday,Hour,Month,load_t-1,load_t-24,load_rolling_24h
utc_timestamp,,,,,,,,,,,,
2015-02-01 00:00:00+00:00,0,6241.0,6186.0,25.0,0.0,131.0,Sunday,0,February,6241.0,6241.0,6967.625000
2015-02-01 01:00:00+00:00,1,6126.0,6111.0,24.0,0.0,119.0,Sunday,1,February,6241.0,6241.0,6967.625000
2015-02-01 02:00:00+00:00,2,5983.0,6019.0,24.0,0.0,89.0,Sunday,2,February,6126.0,6241.0,6967.625000
2015-02-01 03:00:00+00:00,3,5966.0,6027.0,24.0,0.0,56.0,Sunday,3,February,5983.0,6241.0,6967.625000
2015-02-01 04:00:00+00:00,4,6004.0,6247.0,24.0,0.0,56.0,Sunday,4,February,5966.0,6241.0,6967.625000
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-09-30 19:00:00+00:00,32107,6073.0,5774.0,60.0,0.0,1674.0,Sunday,19,September,6533.0,6156.0,5715.500000
2018-09-30 20:00:00+00:00,32108,6091.0,5712.0,58.0,0.0,1773.0,Sunday,20,September,6073.0,6033.0,5717.916667
2018-09-30 21:00:00+00:00,32109,5705.0,5355.0,50.0,0.0,1829.0,Sunday,21,September,6091.0,5612.0,5721.791667


In [82]:
austria_df.drop(columns="Unnamed: 0", inplace= True)

In [83]:
austria_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32112 entries, 0 to 32111
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   utc_timestamp     32112 non-null  object 
 1   load_actual       32112 non-null  float64
 2   load_forecast     32112 non-null  float64
 3   price             32112 non-null  float64
 4   solar_generation  32112 non-null  float64
 5   Wind_generation   32112 non-null  float64
 6   Weekday           32112 non-null  object 
 7   Hour              32112 non-null  int64  
 8   Month             32112 non-null  object 
 9   load_t-1          32112 non-null  float64
 10  load_t-24         32112 non-null  float64
 11  load_rolling_24h  32112 non-null  float64
dtypes: float64(8), int64(1), object(3)
memory usage: 2.9+ MB


### Converting the Weekdays and month

In [84]:
weekday_map = {
    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2,
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6
}

austria_df['Weekday'] = austria_df['Weekday'].map(weekday_map)

In [85]:
Monthly_map = {
    'January': 0,
    'February': 1,
    'March': 2,
    'April': 3,
    'May': 4,
    'June': 5,
    'July': 6,
    'August': 7,
    'September': 8,
    'October': 9,
    'November': 10,
    'December': 11,
}

austria_df['Month'] = austria_df['Month'].map(Monthly_map)

In [86]:
'''
An alternative method to create some feature but we are not using it.

austria_df = pd.get_dummies(austria_df, columns=['Weekday'], drop_first=True)
austria_df = pd.get_dummies(austria_df, columns=['Month'], drop_first=True)

features = [
    'load_forecast', 'solar_generation', 'Wind_generation',
    'Hour', 'load_t-1', 'load_t-24', 'load_rolling_24h'
] + [col for col in austria_df.columns if col.startswith('Weekday_') or col.startswith('Month_')] '''

"\nAn alternative method to create some feature but we are not using it.\n\naustria_df = pd.get_dummies(austria_df, columns=['Weekday'], drop_first=True)\naustria_df = pd.get_dummies(austria_df, columns=['Month'], drop_first=True)\n\nfeatures = [\n    'load_forecast', 'solar_generation', 'Wind_generation',\n    'Hour', 'load_t-1', 'load_t-24', 'load_rolling_24h'\n] + [col for col in austria_df.columns if col.startswith('Weekday_') or col.startswith('Month_')] "

In [87]:
austria_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32112 entries, 0 to 32111
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   utc_timestamp     32112 non-null  object 
 1   load_actual       32112 non-null  float64
 2   load_forecast     32112 non-null  float64
 3   price             32112 non-null  float64
 4   solar_generation  32112 non-null  float64
 5   Wind_generation   32112 non-null  float64
 6   Weekday           32112 non-null  int64  
 7   Hour              32112 non-null  int64  
 8   Month             32112 non-null  int64  
 9   load_t-1          32112 non-null  float64
 10  load_t-24         32112 non-null  float64
 11  load_rolling_24h  32112 non-null  float64
dtypes: float64(8), int64(3), object(1)
memory usage: 2.9+ MB


### Spliting data for train and test

In [88]:
# Training set 2015 and 2016
train_df = austria_df.loc['2015-02-01':'2017-06-31']

# Testing set 2017
test_df = austria_df.loc['2016-07-01':'2018-09-30']

In [89]:
# Features
target = 'price'
features = [
    'load_forecast', 'solar_generation', 'Wind_generation',
    'Hour', 'Month', 'load_t-1', 'load_t-24', 'load_rolling_24h','Weekday']

# Train
X_train = train_df[features]
y_train = train_df[target]

# Test
X_test = test_df[features]
y_test = test_df[target]

### Gradiant Boost Regressor

In [90]:
# Initiating the model
simple_gbr = GradientBoostingRegressor(n_estimators=200, learning_rate=0.02, max_depth=5, random_state=42)

# Fit
simple_gbr.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.02, max_depth=5, n_estimators=200,
                          random_state=42)

In [91]:
# Prediction the test data
y_pred = simple_gbr.predict(X_test)

# Creating the performance metrix
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R2S:", r2_score(y_test, y_pred))

RMSE: 2.290321523351232
R2S: 0.8463961147781578


In [92]:
### Checking why error is high
print(austria_df['price'].describe())

count    32112.000000
mean        33.988883
std         14.888293
min        -76.000000
25%         25.000000
50%         33.000000
75%         42.000000
max        143.000000
Name: price, dtype: float64


### Using Grid search for futurther improving the Model 
using 'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7] and 'learning_rate': [0.01, 0.1, 0.2]

In [ ]:
from sklearn.model_selection import GridSearchCV

